In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import re
import nltk
from nltk import word_tokenize
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch

In [2]:
df=pd.read_csv('C:\\Users\\Gargi Joshi\\Build-your-own-GPT-bot\\week_2_pytorch\\IMDB Dataset.csv')
data=df
data['sentiment']=[1 if s=="positive" else 0 for s in data['sentiment']]

In [55]:
reviews=[]
for i in range(100):
    reviews.append(df["review"][i])

In [56]:
def preprocess(data):
    output=[]
    for text in data:
        text = re.sub(r'\<br\s*/?\>', ' ', text)
        text = re.sub(r"[?']", '', text)
        text = word_tokenize(text.lower())
        output.append(text)
    return(output)
reviews=preprocess(reviews)

In [57]:
vocabulary={'<unk>':0}
for tokens in reviews:
  for token in Counter(tokens).keys():
    if token not in vocabulary:
        vocabulary[token] = len(vocabulary)

In [18]:
len(vocabulary)

153804

In [58]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence


In [67]:
reviews_vectors= []

for sentence in reviews:
  reviews_vectors.append(text_to_indices(sentence, vocabulary))

sentiments=[]
for i in range(100):
    sentiments.append([data["sentiment"][i]])
    

In [60]:
tensor_reviews = [torch.tensor(review, dtype=torch.long) for review in reviews_vectors]
padded_reviews = pad_sequence(tensor_reviews, batch_first=True, padding_value=0)

In [61]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [68]:
#reviews_tensor=torch.from_numpy(padded_reviews, dtype=np.float64)
sentiments_tensor=torch.from_numpy(np.array(sentiments))
dataset = CustomDataset(padded_reviews, sentiments_tensor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [69]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc=nn.Linear(150, 1)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [70]:
model = LSTMModel(len(vocabulary))

In [65]:
#model.to("cpu")
epochs = 50
learning_rate = 0.001

criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [73]:
for epoch in range(epochs):
  total_loss = 0
  correct = 0
  total = 0
  for batch_x, batch_y in dataloader:
      batch_x, batch_y = batch_x.to("cpu"), batch_y.to("cpu").float()
      #batch_y=batch_y.float()
      #print(batch_y.dtype, batch_y.shape)
      optimizer.zero_grad()
      output = model(batch_x)
      #print(output[4], batch_y[4])
      loss = criterion(output, batch_y)

      loss.backward()

      optimizer.step()

      total_loss = total_loss + loss.item()
      predictions = (torch.sigmoid(output) > 0.5).int()  # convert logits to 0/1
      correct += (predictions == batch_y.int()).sum().item()
      total += batch_y.size(0)
      print(correct)

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")
  accuracy = correct / total
  print(accuracy)

4
8
15
20
26
31
34
38
41
45
50
56
58
Epoch: 1, Loss: 8.9224
0.58
5
9
12
17
22
30
35
38
41
46
51
54
58
Epoch: 2, Loss: 8.8949
0.58
4
9
15
20
25
29
33
37
42
47
53
56
58
Epoch: 3, Loss: 8.9220
0.58
4
8
12
16
23
27
32
36
43
47
52
56
58
Epoch: 4, Loss: 8.9220
0.58
4
7
13
18
24
31
34
40
44
48
51
56
58
Epoch: 5, Loss: 8.9220
0.58
7
10
15
19
24
30
33
37
42
47
53
57
58
Epoch: 6, Loss: 8.9356
0.58
3
9
15
20
24
27
33
38
42
45
49
55
58
Epoch: 7, Loss: 8.9085
0.58
4
6
12
17
21
25
31
32
40
46
52
56
58
Epoch: 8, Loss: 8.9220
0.58
3
9
16
22
26
31
36
40
44
47
53
57
58
Epoch: 9, Loss: 8.9356
0.58
4
7
11
15
20
27
31
37
40
42
48
55
58
Epoch: 10, Loss: 8.9085
0.58
5
10
16
20
25
29
35
40
42
45
51
56
58
Epoch: 11, Loss: 8.9220
0.58
7
11
15
19
23
27
31
34
37
44
50
57
58
Epoch: 12, Loss: 8.9356
0.58
6
9
13
20
26
30
34
37
43
49
54
56
58
Epoch: 13, Loss: 8.9220
0.58
5
11
17
21
25
29
36
41
45
48
54
57
58
Epoch: 14, Loss: 8.9356
0.58
4
8
13
18
21
24
28
35
38
44
51
57
58
Epoch: 15, Loss: 8.9356
0.58
5
8
11
14
20
24

In [82]:
def predict_statement(statement_vectors):
    model.eval()
    with torch.no_grad():
        output = model(statement_tensor)
        probs = torch.sigmoid(output)
        return probs
    
string=str(input("Enter review:"))
string_preprocess=preprocess(string)[0]
string_vectors = text_to_indices(string_preprocess, vocabulary)
statement_tensor = torch.tensor(string_vectors, dtype=torch.long).unsqueeze(0)

output = predict_statement(statement_tensor)
predicted_label = (output > 0.5).int().item()
print("Prediction:", "Positive" if predicted_label == 1 else "Negative")
#CLEARLY DOESN'T WORK I DONT KNOW WHY 

Enter review: Amazing


Prediction: Negative
